In [2]:
import os, sys, json, datetime, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
from tqdm import tqdm
import glob as glob

#thi data contants
from cprl_functions.defined_functions import *
from cprl_functions.state_capture import *
from cprl_functions.text_printing import bordered
from cprl_functions.data_packet_defs import *



In [44]:
def clean_str(str):
    cl = str.lower().replace(',','').replace('(','').replace(')','').replace("-","").replace(" ","_").strip()
    return cl

In [17]:
state_ref

{'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY',
 'District of Columbia': 'DC

In [ ]:
def get_state_abv(text):
    pat  = re.compile('|'.join(state_list), re.IGNORECASE)
    match = re.search(pat, text)
    if 'District' in match.group(0):
        return 'DC'
        
    string = match.group(0)
    abv = state_ref.get(string)
    return abv


In [81]:

state_coding

{'AL': 10,
 'AK': 11,
 'AZ': 12,
 'AR': 13,
 'CA': 14,
 'CO': 15,
 'CT': 16,
 'DE': 17,
 'FL': 18,
 'GA': 19,
 'HI': 20,
 'ID': 21,
 'IL': 22,
 'IN': 23,
 'IA': 24,
 'KS': 25,
 'KY': 26,
 'LA': 27,
 'ME': 28,
 'MD': 29,
 'MA': 30,
 'MI': 31,
 'MN': 32,
 'MS': 33,
 'MO': 34,
 'MT': 35,
 'NE': 36,
 'NV': 37,
 'NH': 38,
 'NJ': 39,
 'NM': 40,
 'NY': 41,
 'NC': 42,
 'ND': 43,
 'OH': 44,
 'OK': 45,
 'OR': 46,
 'PA': 47,
 'RI': 48,
 'SC': 49,
 'SD': 50,
 'TN': 51,
 'TX': 52,
 'UT': 53,
 'VT': 54,
 'VA': 55,
 'WA': 56,
 'WV': 57,
 'WI': 58,
 'WY': 59,
 'DC': 60}

In [104]:
def get_state_code(state_abv):
    result = state_coding.get(state_abv)
    # print(state_abv)
    # print(result)
    return str(result)

In [105]:
def pop_column(df, label):
    popped_col = df.pop(label)
    df.insert(0,label,popped_col)
    return df


In [106]:
# clean up
df = pd.read_csv("data_collection_links.csv")
df['state_abv'] =  df['State'].apply(get_state_abv)
st_abv_pop = df.pop('state_abv')
df.insert(0,'state_abv',st_abv_pop)

df.columns = [clean_str(x) for x in df.columns]
mapping = {}
for i,col in enumerate(df.columns):
    if i > 1:
        mapping[col] = f'metric_{col}'

df = df.rename(columns=mapping)
df['state_code'] = df['state_abv'].apply(get_state_code)
new_df = pop_column(df,'state_code')


In [108]:
new_df.to_clipboard()

In [ ]:
# counting unique sources
from collections import Counter
statess_dicts = {}
for row in df.itertuples(index=True):
    print(bordered(row.state))
    # print(row)
    values = []
    to_get_dict = {}
    for col in row._fields:
        if 'metric' not in col:
            continue
        val = getattr(row, col)
        match = re.search(r'https://(.+)\.[govcomorgedus]{2,3}/', val)
        try:
            new_val = match.group(1)
        except:
            print(col,val)
            break
        to_get_dict[col] = new_val
        values.append(new_val)
        # print(f"{col}: {getattr(row, col)}")

    cnt = Counter(values)
    for char, count in cnt.items():
        print(bordered(f'{char}: {count}'))
        for k,v in to_get_dict.items():
            if char==v:
                print(k)

    print(to_get_dict.get(char))

    statess_dicts[row.state_abv] = to_get_dict

┌───────┐
│Alabama│
└───────┘
┌───────────────────┐
│reportcard.alsde: 5│
└───────────────────┘
metric_k12_enrollment_link_text
metric_k12_enrollment_raceethnicity_link_text
metric_k12_enrollment_ses_link_text
metric_chronic_absenteeism_link_text
metric_hs_graduation_rate_link_text
┌────────────┐
│www.alsde: 1│
└────────────┘
metric_state_proficiency_link_text
None
┌──────┐
│Alaska│
└──────┘
┌───────────────────┐
│education.alaska: 6│
└───────────────────┘
metric_k12_enrollment_link_text
metric_k12_enrollment_raceethnicity_link_text
metric_k12_enrollment_ses_link_text
metric_state_proficiency_link_text
metric_chronic_absenteeism_link_text
metric_hs_graduation_rate_link_text
None
┌───────┐
│Arizona│
└───────┘
┌─────────────────────┐
│azreportcards.azed: 4│
└─────────────────────┘
metric_k12_enrollment_link_text
metric_k12_enrollment_raceethnicity_link_text
metric_chronic_absenteeism_link_text
metric_hs_graduation_rate_link_text
┌───────────┐
│www.azed: 2│
└───────────┘
metric_k12_enroll

In [ ]:
Pretty

In [80]:
al_data = pd.read_csv('demographics_2024_al.csv')
al_data

,Year,System Code,System,School Code,School,Grade,Gender,Ethnicity,Sub Population,Total Student Count,...,Black or African American,Black or African American %,American Indian / Alaska Native,American Indian / Alaska Native %,Native Hawaiian / Pacific Islander,Native Hawaiian / Pacific Islander %,White,White %,Two or more races,Two or more races %
0,2024,0,Alabama State Department of Education,0,Alabama State Department of Education,All Grades,All Gender,All Ethnicity,All SubPopulation,718716,...,229512,31.93,30295,~,1911,~,407758,56.73,37946,5.28
1,2024,0,Alabama State Department of Education,0,Alabama State Department of Education,All Grades,All Gender,All Ethnicity,Students with Limited English Proficiency,47838,...,1016,~,15987,33.42,874,~,21321,44.57,5669,11.85
2,2024,0,Alabama State Department of Education,0,Alabama State Department of Education,All Grades,All Gender,All Ethnicity,Homeless,12844,...,6265,48.78,897,6.98,56,~,4410,34.34,1129,8.79
3,2024,0,Alabama State Department of Education,0,Alabama State Department of Education,All Grades,All Gender,All Ethnicity,Migrant,1277,...,29,~,644,50.43,21,~,374,29.29,201,15.74
4,2024,0,Alabama State Department of Education,0,Alabama State Department of Education,All Grades,All Gender,All Ethnicity,Students with Disabilities,129379,...,44144,34.12,3423,~,219,~,74548,57.62,6065,~
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,2024,0,Alabama State Department of Education,0,Alabama State Department of Education,Grade 12,Male,Other Ethnicity,General Education Students,17530,...,5656,32.26,187,~,20,~,10824,61.75,490,~
1330,2024,0,Alabama State Department of Education,0,Alabama State Department of Education,Grade 12,Male,Other Ethnicity,Economically Disadvantaged,11509,...,5696,49.49,99,~,14,~,5197,45.16,357,~
1331,2024,0,Alabama State Department of Education,0,Alabama State Department of Education,Grade 12,Male,Other Ethnicity,Foster,69,...,35,50.72,*,*,*,*,34,49.28,*,*
1332,2024,0,Alabama State Department of Education,0,Alabama State Department of Education,Grade 12,Male,Other Ethnicity,Military Family,602,...,182,30.23,*,*,*,*,370,61.46,24,~


In [78]:
import pprint
for k,v in statess_dicts.items():
    print('______________')
    print(k)
    pprint.pprint(v)

______________
AL
{'metric_chronic_absenteeism_link_text': 'reportcard.alsde',
 'metric_hs_graduation_rate_link_text': 'reportcard.alsde',
 'metric_k12_enrollment_link_text': 'reportcard.alsde',
 'metric_k12_enrollment_raceethnicity_link_text': 'reportcard.alsde',
 'metric_k12_enrollment_ses_link_text': 'reportcard.alsde',
 'metric_state_proficiency_link_text': 'www.alsde'}
______________
AK
{'metric_chronic_absenteeism_link_text': 'education.alaska',
 'metric_hs_graduation_rate_link_text': 'education.alaska',
 'metric_k12_enrollment_link_text': 'education.alaska',
 'metric_k12_enrollment_raceethnicity_link_text': 'education.alaska',
 'metric_k12_enrollment_ses_link_text': 'education.alaska',
 'metric_state_proficiency_link_text': 'education.alaska'}
______________
AZ
{'metric_chronic_absenteeism_link_text': 'azreportcards.azed',
 'metric_hs_graduation_rate_link_text': 'azreportcards.azed',
 'metric_k12_enrollment_link_text': 'azreportcards.azed',
 'metric_k12_enrollment_raceethnicity_